In [1]:
%cd mmdetection
import mmcv

/data/Detection_proj/mmdetection


In [15]:
from mmdet.apis import set_random_seed
from mmcv import Config
from pprint import pprint
#cfg = Config.fromfile('./configs/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_1x_coco.py')
#faster_rcnn_r50_caffe_fpn_1x_coco_bbox_mAP-0.378_20200504_180032-c5925ee5.pth
cfg = Config.fromfile('./configs/ssd/ssd512_coco.py')
print(f'Config:\n{cfg.pretty_text}')

Config:
input_size = 512
model = dict(
    type='SingleStageDetector',
    backbone=dict(
        type='SSDVGG',
        depth=16,
        with_last_pool=False,
        ceil_mode=True,
        out_indices=(3, 4),
        out_feature_indices=(22, 34),
        init_cfg=dict(
            type='Pretrained', checkpoint='open-mmlab://vgg16_caffe')),
    neck=dict(
        type='SSDNeck',
        in_channels=(512, 1024),
        out_channels=(512, 1024, 512, 256, 256, 256, 256),
        level_strides=(2, 2, 2, 2, 1),
        level_paddings=(1, 1, 1, 1, 1),
        l2_norm_scale=20,
        last_kernel_size=4),
    bbox_head=dict(
        type='SSDHead',
        in_channels=(512, 1024, 512, 256, 256, 256, 256),
        num_classes=80,
        anchor_generator=dict(
            type='SSDAnchorGenerator',
            scale_major=False,
            input_size=512,
            basesize_ratio_range=(0.1, 0.9),
            strides=[8, 16, 32, 64, 128, 256, 512],
            ratios=[[2], [2, 3], [2, 

In [16]:
import os
import os.path as osp
CLASS = ['china','us','uk','russia','japan','france','german','italy','australia','korea','other']
                
# Modify dataset type and path
cfg.data_root = '/data/flag/Images'
cfg.classes = CLASS

cfg.data.samples_per_gpu = 2
cfg.data.workers_per_gpu = 4

cfg.data.test.img_prefix = '/data/flag/Images'
cfg.data.test.ann_file = '/data/flag/flag_coco_test.json'
cfg.data.test.classes = CLASS

cfg.data.train.dataset.ann_file = '/data/flag/flag_coco_train.json'
cfg.data.train.dataset.img_prefix = '/data/flag/Images'
cfg.data.train.dataset.classes = CLASS

cfg.data.val.img_prefix = '/data/flag/Images'
cfg.data.val.ann_file = '/data/flag/flag_coco_val.json'
cfg.data.val.classes = CLASS

# modify num classes of the model in box head
cfg.model.bbox_head.num_classes = 11
#cfg.model.roi_head.mask_head.num_classes = 1
# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
#cfg.load_from = 'checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_\
#coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth'
cfg.load_from = 'checkpoints/ssd512_coco_20210604_111835-d3eba047.pth'
# Set up working dir to save files and logs.
cfg.work_dir = './flag_proj/ssd'

if not os.path.exists(cfg.work_dir):
    os.makedirs(cfg.work_dir)
# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.002 / 4
#cfg.lr_config.warmup = None
#log interval by step not epoch
cfg.log_config.interval = 100

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = ['bbox']
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 1
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 1

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

#cfg.workflow = [('train',1),('val',1)]

#cfg.runner.max_epochs = 1

# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')
cfg.dump(osp.join(cfg.work_dir, 'flag_config.py'))

Config:
input_size = 512
model = dict(
    type='SingleStageDetector',
    backbone=dict(
        type='SSDVGG',
        depth=16,
        with_last_pool=False,
        ceil_mode=True,
        out_indices=(3, 4),
        out_feature_indices=(22, 34),
        init_cfg=dict(
            type='Pretrained', checkpoint='open-mmlab://vgg16_caffe')),
    neck=dict(
        type='SSDNeck',
        in_channels=(512, 1024),
        out_channels=(512, 1024, 512, 256, 256, 256, 256),
        level_strides=(2, 2, 2, 2, 1),
        level_paddings=(1, 1, 1, 1, 1),
        l2_norm_scale=20,
        last_kernel_size=4),
    bbox_head=dict(
        type='SSDHead',
        in_channels=(512, 1024, 512, 256, 256, 256, 256),
        num_classes=11,
        anchor_generator=dict(
            type='SSDAnchorGenerator',
            scale_major=False,
            input_size=512,
            basesize_ratio_range=(0.1, 0.9),
            strides=[8, 16, 32, 64, 128, 256, 512],
            ratios=[[2], [2, 3], [2, 

In [17]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
import os.path as osp
import copy

# Build dataset
datasets = [build_dataset(cfg.data.train)]
if len(cfg.workflow) == 2:
    val_dataset = copy.deepcopy(cfg.data.val)
    val_dataset.pipeline = cfg.train_pipeline
    datasets.append(build_dataset(val_dataset))
# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


In [19]:
datasets[0]

In [20]:
model

SingleStageDetector(
  (backbone): SSDVGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=Tr

In [21]:
train_detector(model, datasets, cfg, distributed=False, validate=True)

2021-07-06 12:41:18,593 - mmdet - INFO - load checkpoint from checkpoints/ssd512_coco_20210604_111835-d3eba047.pth
2021-07-06 12:41:18,594 - mmdet - INFO - Use load_from_local loader
2021-07-06 12:41:18,692 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.cls_convs.0.0.weight: copying a param with shape torch.Size([324, 512, 3, 3]) from checkpoint, the shape in current model is torch.Size([48, 512, 3, 3]).
size mismatch for bbox_head.cls_convs.0.0.bias: copying a param with shape torch.Size([324]) from checkpoint, the shape in current model is torch.Size([48]).
size mismatch for bbox_head.cls_convs.1.0.weight: copying a param with shape torch.Size([486, 1024, 3, 3]) from checkpoint, the shape in current model is torch.Size([72, 1024, 3, 3]).
size mismatch for bbox_head.cls_convs.1.0.bias: copying a param with shape torch.Size([486]) from checkpoint, the shape in current model is torch.Size([72]).
size mismatch for bbox_head.cls_convs

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


/data/Detection_proj/mmdetection/mmdet/core/anchor/anchor_generator.py:323: UserWarning: ``grid_anchors`` would be deprecated soon. Please use ``grid_priors`` 
  warnings.warn('``grid_anchors`` would be deprecated soon. '
/data/Detection_proj/mmdetection/mmdet/core/anchor/anchor_generator.py:360: UserWarning: ``single_level_grid_anchors`` would be deprecated soon. Please use ``single_level_grid_priors`` 
  '``single_level_grid_anchors`` would be deprecated soon. '
2021-07-06 12:42:05,757 - mmdet - INFO - Epoch [1][100/6318]	lr: 9.940e-05, eta: 19:41:12, time: 0.468, data_time: 0.027, memory: 1355, loss_cls: 10.8389, loss_bbox: 1.9622, loss: 12.8011
2021-07-06 12:42:50,278 - mmdet - INFO - Epoch [1][200/6318]	lr: 1.993e-04, eta: 19:12:04, time: 0.445, data_time: 0.005, memory: 1355, loss_cls: 6.6531, loss_bbox: 1.7766, loss: 8.4298
2021-07-06 12:43:34,832 - mmdet - INFO - Epoch [1][300/6318]	lr: 2.992e-04, eta: 19:02:09, time: 0.446, data_time: 0.005, memory: 1355, loss_cls: 4.7581, los

2021-07-06 13:13:32,284 - mmdet - INFO - Epoch [1][4300/6318]	lr: 5.000e-04, eta: 18:24:00, time: 0.450, data_time: 0.005, memory: 1355, loss_cls: 3.0056, loss_bbox: 1.3332, loss: 4.3388
2021-07-06 13:14:17,229 - mmdet - INFO - Epoch [1][4400/6318]	lr: 5.000e-04, eta: 18:23:14, time: 0.449, data_time: 0.005, memory: 1355, loss_cls: 3.0551, loss_bbox: 1.2810, loss: 4.3361
2021-07-06 13:15:02,187 - mmdet - INFO - Epoch [1][4500/6318]	lr: 5.000e-04, eta: 18:22:29, time: 0.450, data_time: 0.005, memory: 1355, loss_cls: 2.9065, loss_bbox: 1.3345, loss: 4.2410
2021-07-06 13:15:47,127 - mmdet - INFO - Epoch [1][4600/6318]	lr: 5.000e-04, eta: 18:21:44, time: 0.449, data_time: 0.005, memory: 1355, loss_cls: 2.9445, loss_bbox: 1.2256, loss: 4.1701
2021-07-06 13:16:32,087 - mmdet - INFO - Epoch [1][4700/6318]	lr: 5.000e-04, eta: 18:20:59, time: 0.450, data_time: 0.005, memory: 1355, loss_cls: 2.8971, loss_bbox: 1.1709, loss: 4.0680
2021-07-06 13:17:17,174 - mmdet - INFO - Epoch [1][4800/6318]	lr:

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 361/361, 11.0 task/s, elapsed: 33s, ETA:     0s

...

Loading and preparing results...
DONE (t=0.19s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=4.10s).
Accumulating evaluation results...


2021-07-06 15:53:28,029 - mmdet - INFO - Epoch(val) [4][361]	bbox_mAP: 0.4040, bbox_mAP_50: 0.6670, bbox_mAP_75: 0.4270, bbox_mAP_s: 0.4270, bbox_mAP_m: 0.4290, bbox_mAP_l: 0.1620, bbox_mAP_copypaste: 0.404 0.667 0.427 0.427 0.429 0.162


DONE (t=1.38s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.404
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.667
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.427
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.427
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.429
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.162
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.485
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.485
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.485
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.502
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.513
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.193


2021-07-06 15:54:15,087 - mmdet - INFO - Epoch [5][100/6318]	lr: 5.000e-04, eta: 15:43:54, time: 0.468, data_time: 0.025, memory: 1355, loss_cls: 2.2163, loss_bbox: 0.8480, loss: 3.0643
2021-07-06 15:54:59,902 - mmdet - INFO - Epoch [5][200/6318]	lr: 5.000e-04, eta: 15:43:09, time: 0.448, data_time: 0.005, memory: 1355, loss_cls: 2.1324, loss_bbox: 0.8354, loss: 2.9677
2021-07-06 15:55:44,724 - mmdet - INFO - Epoch [5][300/6318]	lr: 5.000e-04, eta: 15:42:24, time: 0.448, data_time: 0.005, memory: 1355, loss_cls: 2.2133, loss_bbox: 0.8586, loss: 3.0719
2021-07-06 15:56:29,559 - mmdet - INFO - Epoch [5][400/6318]	lr: 5.000e-04, eta: 15:41:39, time: 0.448, data_time: 0.005, memory: 1355, loss_cls: 2.1985, loss_bbox: 0.9061, loss: 3.1046
2021-07-06 15:57:14,405 - mmdet - INFO - Epoch [5][500/6318]	lr: 5.000e-04, eta: 15:40:54, time: 0.448, data_time: 0.005, memory: 1355, loss_cls: 2.2063, loss_bbox: 0.8649, loss: 3.0712
2021-07-06 15:57:59,330 - mmdet - INFO - Epoch [5][600/6318]	lr: 5.000

KeyboardInterrupt: 